<h1>SVD decomposition with SciPy</h1>

In [3]:
#Import the required modules
import scipy.linalg as la
import numpy as np

#Generate a 5x4 matrix
np.random.seed(0) #Generate the same random numbers each time
A = np.random.rand(5,4)

Input: a matrix $A$ of size $m \times n$. <br>
Oputput: U of size $m \times m$, S of size $m \times n$ and V of size $n \times n$ <br>
or, more precisely,<br>
σ of size q<br>
where q = min(m,n)

In [17]:
U, s, VT = np.linalg.svd(A)
print("U shape", U.shape)
print("s shape", s.shape)
print("VT shape", VT.shape)

U shape (5, 5)
s shape (4,)
VT shape (4, 4)


In [18]:
#Build the matrix S
S = np.zeros(A.shape)
for i in range(len(s)):
    S[i,i] = s[i]
S

array([[2.64618677, 0.        , 0.        , 0.        ],
       [0.        , 0.83351254, 0.        , 0.        ],
       [0.        , 0.        , 0.70753001, 0.        ],
       [0.        , 0.        , 0.        , 0.29842614],
       [0.        , 0.        , 0.        , 0.        ]])

In [19]:
S = la.diagsvd(s, A.shape[0], A.shape[1])
S

array([[2.64618677, 0.        , 0.        , 0.        ],
       [0.        , 0.83351254, 0.        , 0.        ],
       [0.        , 0.        , 0.70753001, 0.        ],
       [0.        , 0.        , 0.        , 0.29842614],
       [0.        , 0.        , 0.        , 0.        ]])

In [20]:
#Reconstruct the matrix A
A_svd = np.matmul(U, np.matmul(S, VT))
#Equivalently: A_svd = U @ S @ VT
print(f"err: {(la.norm(A-A_svd)/la.norm(A))}")

err: 4.2743316456409587e-16


<h3>A note on vectorization</h3>
Vectorization refers to the practice of replacing explicit loops with high-level mathematical operations that act on entire arrays of matrices at once. This leads to much better performance because it replaces slow Python loops with fast, optimized C and Fortran operations.<br>
Indeed, we could be inclined to reconstruct A_k with a for loop and the explicit formula

$$
A_k = \sum_{i=1}^{k} \sigma_{k} u_{k} v_{k}^{T}
$$

Let's measure the time taken for this operation for a matrix A that is a bit larger.

In [24]:
import time

A = np.random.rand(500, 400)
U, s, VT = la.svd(A, full_matrices=False)
S = np.diag(s)

#Time the reconstruction with a for loop
start_time = time.time()
A_reconstructed_loop = np.zeros_like(A) #Initialize a matrix of zeros
for i in range(len(s)):
    A_reconstructed_loop += s[i] * np.outer(U[:,i], VT[i,:])
loop_time = time.time() - start_time

#Time the reconstruction using matrix multiplication
start_time = time.time()
#Here we are using broadcasting to avoid the creation of a diagonal matrix
A_reconstructed_vectorized = (U * s) @ VT
vectorized_time = time.time() - start_time

#We compare the results
print(f"Time for reconstruction using for loop: {loop_time:.6f} seconds")
print(f"Time for vectorized reconstruction: {vectorized_time:.6f} seconds")
print(f"Vectorized is {loop_time/vectorized_time:.1f} times faster than the loop")

difference = np.abs(A_reconstructed_loop - A_reconstructed_vectorized).max()
print(f"Max difference between the two reconstructions: {difference:.6e}")

Time for reconstruction using for loop: 0.107924 seconds
Time for vectorized reconstruction: 0.000752 seconds
Vectorized is 143.5 times faster than the loop
Max difference between the two reconstructions: 8.881784e-16
